In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,16579
2,Huelva,Confirmados PDIA 14 días,4325
3,Huelva,Tasa PDIA 14 días,"846,8055362481717"
4,Huelva,Confirmados PDIA 7 días,2363
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,20
629,Municipio de Huelva sin especificar,Total Confirmados,248
630,Municipio de Huelva sin especificar,Curados,116


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  16579.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  4543.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 282 personas en los últimos 7 días 

Un positivo PCR cada 136 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,16579.0,2363.0,4325.0,510743.0,462.659302,846.805536,591.0
Huelva-Costa,9957.0,1619.0,2934.0,288115.0,561.928397,1018.343370,438.0
Huelva (capital),4543.0,510.0,1056.0,143663.0,354.997459,735.053563,127.0
Condado-Campiña,4512.0,445.0,858.0,155057.0,286.991235,553.344899,121.0
Isla Cristina,1356.0,441.0,759.0,21264.0,2073.927765,3569.413093,116.0
Ayamonte,634.0,178.0,261.0,20946.0,849.804259,1246.061300,60.0
Lepe,1312.0,162.0,282.0,27431.0,590.572710,1028.033976,50.0
Sierra de Huelva-Andévalo Central,1863.0,279.0,443.0,67571.0,412.899025,655.606695,31.0
Punta Umbría,339.0,55.0,84.0,15242.0,360.845033,551.108778,23.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,104.0,42.0,90.0,1305.0,3218.390805,6896.551724,0.0
Villalba del Alcor,299.0,85.0,185.0,3338.0,2546.434991,5542.240863,18.0
Villanueva de las Cruces,15.0,8.0,15.0,373.0,2144.772118,4021.447721,0.0
Isla Cristina,1356.0,441.0,759.0,21264.0,2073.927765,3569.413093,116.0
Calañas,125.0,53.0,75.0,2791.0,1898.960946,2687.208886,4.0
...,...,...,...,...,...,...,...
Alájar,6.0,1.0,1.0,756.0,132.275132,132.275132,0.0
Lucena del Puerto,116.0,1.0,4.0,3371.0,29.664788,118.659152,NaN
Paterna del Campo,73.0,1.0,3.0,3509.0,28.498148,85.494443,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,116.0,1.0,4.0,3371.0,29.664788,118.659152,0.0,0.250000
Cortegana,167.0,8.0,32.0,4666.0,171.453065,685.812259,0.0,0.250000
Bonares,95.0,6.0,23.0,6058.0,99.042588,379.663255,1.0,0.260870
Alosno,89.0,11.0,38.0,3923.0,280.397655,968.646444,2.0,0.289474
Paterna del Campo,73.0,1.0,3.0,3509.0,28.498148,85.494443,0.0,0.333333
...,...,...,...,...,...,...,...,...
Alájar,6.0,1.0,1.0,756.0,132.275132,132.275132,0.0,1.000000
Cumbres Mayores,23.0,3.0,3.0,1755.0,170.940171,170.940171,2.0,1.000000
Cortelazor,3.0,1.0,1.0,299.0,334.448161,334.448161,0.0,1.000000
